# Anomalous Financial Transaction Detection

본 대회의 과제는 금융 거래 데이터에서 **이상 거래를 탐지하는 기능**을 개선하고 활용도를 높이는 분류 AI모델을 개발하는 것입니다. 

특히, 클래스 불균형 문제를 해결하기 위해 오픈소스 생성형 AI 모델을 활용하여 부족한 클래스의 데이터를 보완하고, 이를 통해 분류 모델의 성능을 향상시키는 것이 핵심 목표입니다. 

이러한 접근을 통해 금융보안에 특화된 데이터 분석 및 활용 역량을 강화하여 전문 인력을 양성하고, 금융권의 AI 활용 어려움에 따른 해결 방안을 함께 모색하며 금융 산업의 AI 활용 활성화를 지원하는 것을 목표로 합니다.

# Import Library

In [1]:
# 제출 파일 생성 관련
import os
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

# 머신러닝 전처리
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# 머신러닝 모델
import xgboost as xgb

# 합성 데이터 생성
from sdv.metadata import SingleTableMetadata
from sdv.single_table import TVAESynthesizer


# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')

# 생성 🏭

# Load Data

In [2]:
train_all = pd.read_csv("/workspace/Dataset/FSI/train.csv")
test_all = pd.read_csv("/workspace/Dataset/FSI/test.csv")

In [3]:
train = train_all.drop(columns="ID")

In [4]:
train["Fraud_Type"].value_counts()

m    118800
a       100
j       100
h       100
k       100
c       100
g       100
i       100
b       100
f       100
d       100
e       100
l       100
Name: Fraud_Type, dtype: int64

In [5]:
'''
(*) 리더보드 산식 중 생성데이터의 익명성(TCAP)채점을 위해 각 클래스 별로 1000개의 생성데이터가 반드시 필요합니다.
(*) 본 베이스 라인에서는 "Fraud_Type" 13종류에 대해 1000개씩 , 총 13,000개의 데이터를 생성할 예정입니다.
(*) 분류 모델 성능 개선을 위해 생성 데이터를 활용하는 것에는 생성 데이터의 Row 개수에 제한이 없습니다. 단, 리더보드 평가를 위해 제출을 하는 생성 데이터 프레임은 익명성(TCAP) 평가를 위함이며, 위의 조건을 갖춘 생성 데이터를 제출해야합니다.
'''
N_CLS_PER_GEN = 1000


In [6]:
'''
handle outlier generate 전에 안하기

'''

'\nhandle outlier generate 전에 안하기\n\n'

In [7]:

# 이상치 처리 함수
def handle_outliers(series, n_std=3):
    mean = series.mean()
    std = series.std()
    z_scores = np.abs(stats.zscore(series))
    return series.mask(z_scores > n_std, mean)

# Time_difference 컬럼을 총 초로 변환 및 이상치 처리
train['Time_difference_seconds'] = pd.to_timedelta(train['Time_difference']).dt.total_seconds()
# train['Time_difference_seconds'] = handle_outliers(train['Time_difference_seconds'])


# 모든 Fraud_Type 목록 생성 (m 포함)
fraud_types = train['Fraud_Type'].unique()

# 모든 합성 데이터를 저장할 DataFrame 초기화
all_synthetic_data = pd.DataFrame()

N_SAMPLE = 100

# 각 Fraud_Type에 대해 합성 데이터 생성 및 저장
for fraud_type in tqdm(fraud_types):
    
    # 해당 Fraud_Type에 대한 서브셋 생성
    subset = train[train["Fraud_Type"] == fraud_type]

    #m은 샘플링 더 하기
    if fraud_type == "m":
        N_SAMPLE = 1000
    else:
        N_SAMPLE = 100

    # 모든 Fraud_Type에 대해 100개씩 샘플링
    subset = subset.sample(n=N_SAMPLE, random_state=77)
    
    # Time_difference 열 제외 (초 단위로 변환된 컬럼만 사용)
    subset = subset.drop('Time_difference', axis=1)
    
    # 메타데이터 생성 및 모델 학습
    metadata = SingleTableMetadata()

    metadata.detect_from_dataframe(subset)
    metadata.set_primary_key(None)

    # 데이터 타입 설정
    column_sdtypes = {
        'Account_initial_balance': 'numerical',
        'Account_balance': 'numerical',
        'Customer_identification_number': 'categorical',  
        'Customer_personal_identifier': 'categorical',
        'Account_account_number': 'categorical',
        'IP_Address': 'ipv4_address',  
        'Location': 'categorical',
        'Recipient_Account_Number': 'categorical',
        'Fraud_Type': 'categorical',
        'Time_difference_seconds': 'numerical',
        'Customer_Birthyear': 'numerical'
    }

    # 각 컬럼에 대해 데이터 타입 설정
    for column, sdtype in column_sdtypes.items():
        metadata.update_column(
            column_name=column,
            sdtype=sdtype
        )
        
    synthesizer = TVAESynthesizer(
        metadata,
        epochs=1000,
        batch_size=32,
        embedding_dim=256,
        compress_dims=(256, 128, 64),
        decompress_dims=(64, 128, 256),
        l2scale=1e-4,
        loss_factor=2
    )

    synthesizer.fit(subset)

    synthetic_subset = synthesizer.sample(num_rows=N_CLS_PER_GEN)
    
    # 생성된 Time_difference_seconds의 이상치 처리
    synthetic_subset['Time_difference_seconds'] = handle_outliers(synthetic_subset['Time_difference_seconds'])
    
    # Time_difference_seconds를 다시 timedelta로 변환
    synthetic_subset['Time_difference'] = pd.to_timedelta(synthetic_subset['Time_difference_seconds'], unit='s')
    
    # Time_difference_seconds 컬럼 제거
    synthetic_subset = synthetic_subset.drop('Time_difference_seconds', axis=1)
    
    # 생성된 데이터를 all_synthetic_data에 추가
    all_synthetic_data = pd.concat([all_synthetic_data, synthetic_subset], ignore_index=True)
# 최종 결과 확인
print("\nFinal All Synthetic Data Shape:", all_synthetic_data.shape)

100%|██████████| 13/13 [21:00<00:00, 96.98s/it]  


Final All Synthetic Data Shape: (13000, 63)


In [8]:
all_synthetic_data.head()

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,Time_difference
0,1960,male,이미숙,nyCCXv-TncGWaS,2004-09-24 10:10:32,C,1,1,0,1,...,2012-05-13 14:50:48,0,0,1,0,1,0,m,2010-10-10 12:51:40,0 days 00:01:42
1,1958,male,나민준,tXjyoP-ZlorpbU,2003-09-05 17:18:35,B,1,1,1,1,...,2022-05-17 04:34:23,0,0,0,1,1,0,m,2027-06-03 20:03:52,3 days 08:00:48
2,1958,male,윤예준,dmzdhr-Ipuuaga,2011-04-28 10:31:49,C,0,0,0,1,...,2005-07-28 16:25:29,0,0,1,0,3,0,m,2013-07-17 04:03:42,4 days 03:24:41
3,1960,female,이서현,SMAcyA-BzAWMnV,2007-12-10 11:52:39,C,0,1,1,1,...,2010-02-08 05:46:02,1,0,0,1,0,0,m,2014-02-28 17:59:16,0 days 00:01:42
4,1955,female,김지혜,UlNfQi-rsSrbOt,2006-07-07 19:42:05,B,1,1,1,1,...,2025-04-01 01:48:53,1,0,0,1,1,0,m,2029-11-21 00:49:08,0 days 00:01:42


## 원본 데이터와 concat

In [9]:
origin_train = train_all.drop(columns="ID")
train_total = pd.concat([origin_train, all_synthetic_data])
train_total.shape

(133000, 63)

# Data Preprocessing 1 : Select x, y

In [10]:
train_x = train_total.drop(columns=['Fraud_Type'])
train_y = train_total['Fraud_Type']

test_x = test_all.drop(columns=['ID'])

# Data Preprocessing 2 : 범주형 변수 인코딩

In [11]:
le_subclass = LabelEncoder()
train_y_encoded = le_subclass.fit_transform(train_y)

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: a, 변환된 숫자: 0
원래 레이블: b, 변환된 숫자: 1
원래 레이블: c, 변환된 숫자: 2
원래 레이블: d, 변환된 숫자: 3
원래 레이블: e, 변환된 숫자: 4
원래 레이블: f, 변환된 숫자: 5
원래 레이블: g, 변환된 숫자: 6
원래 레이블: h, 변환된 숫자: 7
원래 레이블: i, 변환된 숫자: 8
원래 레이블: j, 변환된 숫자: 9
원래 레이블: k, 변환된 숫자: 10
원래 레이블: l, 변환된 숫자: 11
원래 레이블: m, 변환된 숫자: 12


In [12]:
# train_x
# 'Time_difference' 열을 문자열로 변환
train_x['Time_difference'] = train_x['Time_difference'].astype(str)

# 범주형 변수 인코딩
categorical_columns = train_x.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# 훈련 데이터 인코딩
train_x_encoded = train_x.copy()
train_x_encoded[categorical_columns] = ordinal_encoder.fit_transform(train_x[categorical_columns])

# 특성 순서 저장
feature_order = train_x_encoded.columns.tolist()

# Model Define

In [13]:
from autogluon.tabular import TabularPredictor


train_data = train_x_encoded.copy()
train_data['Fraud_Type'] = train_y_encoded

# Initialize the AutoGluon TabularPredictor
predictor = TabularPredictor(label='Fraud_Type', eval_metric='f1_macro').fit(train_data)



No path specified. Models will be saved in: "AutogluonModels/ag-20240826_110819"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          28
Memory Avail:       24.56 GB / 31.23 GB (78.7%)
Disk Space Avail:   885.81 GB / 1006.85 GB (88.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training 

In [14]:
# 테스트 데이터 인코딩
test_x_encoded = test_x.copy()
test_x_encoded[categorical_columns] = ordinal_encoder.transform(test_x[categorical_columns])


# 특성 순서 맞추기 및 데이터 타입 일치
test_x_encoded = test_x_encoded[feature_order]
for col in feature_order:
    test_x_encoded[col] = test_x_encoded[col].astype(train_x_encoded[col].dtype)

In [15]:
# Predict on test data
predictions = predictor.predict(test_x_encoded)

# Reverse transform to get original labels if necessary
predictions_label = le_subclass.inverse_transform(predictions)

# Submission

In [16]:
# 분류 예측 결과 제출 데이터프레임(DataFrame)
# 분류 예측 결과 데이터프레임 파일명을 반드시 clf_submission.csv 로 지정해야합니다.
clf_submission = pd.read_csv("/workspace/Dataset/FSI/sample_submission.csv")
clf_submission["Fraud_Type"] = predictions_label
clf_submission.head()

,ID,Fraud_Type
0,TEST_000000,m
1,TEST_000001,m
2,TEST_000002,m
3,TEST_000003,m
4,TEST_000004,m


In [17]:
# 합성 데이터 생성 결과 제출 데이터프레임(DataFrame)
# 합성 데이터 생성 결과 데이터프레임 파일명을 반드시 syn_submission.csv 로 지정해야합니다.
all_synthetic_data.head()

,Customer_Birthyear,Customer_Gender,Customer_personal_identifier,Customer_identification_number,Customer_registration_datetime,Customer_credit_rating,Customer_flag_change_of_authentication_1,Customer_flag_change_of_authentication_2,Customer_flag_change_of_authentication_3,Customer_flag_change_of_authentication_4,...,Last_bank_branch_transaction_datetime,Flag_deposit_more_than_tenMillion,Unused_account_status,Recipient_account_suspend_status,Number_of_transaction_with_the_account,Transaction_history_with_the_account,First_time_iOS_by_vulnerable_user,Fraud_Type,Transaction_resumed_date,Time_difference
0,1960,male,이미숙,nyCCXv-TncGWaS,2004-09-24 10:10:32,C,1,1,0,1,...,2012-05-13 14:50:48,0,0,1,0,1,0,m,2010-10-10 12:51:40,0 days 00:01:42
1,1958,male,나민준,tXjyoP-ZlorpbU,2003-09-05 17:18:35,B,1,1,1,1,...,2022-05-17 04:34:23,0,0,0,1,1,0,m,2027-06-03 20:03:52,3 days 08:00:48
2,1958,male,윤예준,dmzdhr-Ipuuaga,2011-04-28 10:31:49,C,0,0,0,1,...,2005-07-28 16:25:29,0,0,1,0,3,0,m,2013-07-17 04:03:42,4 days 03:24:41
3,1960,female,이서현,SMAcyA-BzAWMnV,2007-12-10 11:52:39,C,0,1,1,1,...,2010-02-08 05:46:02,1,0,0,1,0,0,m,2014-02-28 17:59:16,0 days 00:01:42
4,1955,female,김지혜,UlNfQi-rsSrbOt,2006-07-07 19:42:05,B,1,1,1,1,...,2025-04-01 01:48:53,1,0,0,1,1,0,m,2029-11-21 00:49:08,0 days 00:01:42


In [18]:
'''
(*) 저장 시 각 파일명을 반드시 확인해주세요.
    1. 분류 예측 결과 데이터프레임 파일명 = clf_submission.csv
    2. 합성 데이터 생성 결과 데이터프레임 파일명 = syn_submission.csv

(*) 제출 파일(zip) 내에 두 개의 데이터프레임이 각각 위의 파일명으로 반드시 존재해야합니다.
(*) 파일명을 일치시키지 않으면 채점이 불가능합니다.
'''

# 폴더 생성 및 작업 디렉토리 변경
os.makedirs('/workspace/Dataset/FSI/tvae_finetuen+autogluon', exist_ok=True)
os.chdir("/workspace/Dataset/FSI/tvae_finetuen+autogluon")

# CSV 파일로 저장
clf_submission.to_csv('/workspace/Dataset/FSI/tvae_finetuen+autogluon/clf_submission.csv', encoding='UTF-8-sig', index=False)
all_synthetic_data.to_csv('/workspace/Dataset/FSI/tvae_finetuen+autogluon/syn_submission.csv', encoding='UTF-8-sig', index=False)

# ZIP 파일 생성 및 CSV 파일 추가
with zipfile.ZipFile("/workspace/Dataset/FSI/tvae_finetuen+autogluon/baseline_submission.zip", 'w') as submission:
    submission.write('clf_submission.csv')
    submission.write('syn_submission.csv')
    
print('Done.')

Done.
